### Import Required Libraries and Set Up Environment Variables

In [3]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

### CME Data

In [6]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
query_url= f"{base_url}{CME}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"

In [8]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response=requests.get(query_url)

In [10]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json=cme_response.json()

In [12]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format datap[r

print(json.dumps(cme_json[:1], indent=4))

[
    {
        "activityID": "2013-05-01T03:12:00-CME-001",
        "catalog": "M2M_CATALOG",
        "startTime": "2013-05-01T03:12Z",
        "instruments": [
            {
                "displayName": "SOHO: LASCO/C2"
            },
            {
                "displayName": "SOHO: LASCO/C3"
            },
            {
                "displayName": "STEREO A: SECCHI/COR2"
            },
            {
                "displayName": "STEREO B: SECCHI/COR2"
            }
        ],
        "sourceLocation": "",
        "activeRegionNum": null,
        "note": "",
        "submissionTime": "2013-08-07T16:54Z",
        "versionId": 1,
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
        "cmeAnalyses": [
            {
                "isMostAccurate": true,
                "time21_5": "2013-05-01T07:07Z",
                "latitude": 12.0,
                "longitude": -120.0,
                "halfAngle": 36.0,
                "speed": 860.0,
        

In [14]:
# Convert cme_json to a Pandas DataFrame 
cme_df=pd.DataFrame(cme_json)

# Keep only the columns: activityID, startTime, linkedEvents
cme_df=cme_df[['activityID','startTime','linkedEvents']]
cme_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,2013-05-02T05:24Z,None
2,2013-05-02T14:36:00-CME-001,2013-05-02T14:36Z,None
3,2013-05-03T18:00:00-CME-001,2013-05-03T18:00Z,None
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]


In [16]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df.dropna(subset='linkedEvents',inplace=True)
cme_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...


In [18]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows=[]

# Iterate over each index in the DataFrame
for i in cme_df.index: 
    activityID=cme_df.loc[i,'activityID']
    startTime=cme_df.loc[i,'startTime']
    linkedEvents=cme_df.loc[i,'linkedEvents']
    
    # Iterate over each dictionary in the list
    for item in linkedEvents:
        expanded_rows.append({'activityID':activityID,'startTime':startTime,'linkedEvents':item})
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
      
# Create a new DataFrame from the expanded rows
expanded_rows_df=pd.DataFrame(expanded_rows)
expanded_rows_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'}
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'}
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'}
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'}
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'}


In [20]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
def extract_activityID_from_dict(input_dict):
    try:
        activityID=input_dict.get('activityID',None)
        
        return activityID
        
    except (ValueError, TypeError) as e:
        print("Error")
        return None
        # Log the error or print it for debugging

extract_activityID_from_dict(expanded_rows_df.loc[0,'linkedEvents'])

'2013-05-04T04:52:00-IPS-001'

In [22]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
expanded_rows_df['GST_ActivityID']=expanded_rows_df['linkedEvents'].apply(lambda x:extract_activityID_from_dict(x))
expanded_rows_df.head()

,activityID,startTime,linkedEvents,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001


In [24]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
expanded_rows_df.dropna(subset='GST_ActivityID',inplace=True)
expanded_rows_df.head()

,activityID,startTime,linkedEvents,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001


In [26]:
# print out the datatype of each column in this DataFrame:
expanded_rows_df.dtypes

activityID        object
startTime         object
linkedEvents      object
GST_ActivityID    object
dtype: object

In [28]:
# Convert the 'GST_ActivityID' column to string format 
expanded_rows_df['GST_ActivityID']=expanded_rows_df['GST_ActivityID'].astype(str)

# Convert startTime to datetime format  
expanded_rows_df['startTime']=pd.to_datetime(expanded_rows_df['startTime'])

# Rename startTime to startTime_CME and activityID to cmeID
expanded_rows_df.rename(columns={'startTime':'startTime_CME','activityID':'cmeID'},inplace=True)

# Drop linkedEvents
expanded_rows_df.drop(columns='linkedEvents', inplace=True)

# Verify that all steps were executed correctly
expanded_rows_df.columns

Index(['cmeID', 'startTime_CME', 'GST_ActivityID'], dtype='object')

In [30]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
expanded_rows_df=expanded_rows_df[expanded_rows_df['GST_ActivityID'].str.contains('GST')]
expanded_rows_df.head()

,cmeID,startTime_CME,GST_ActivityID
21,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
48,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
90,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
148,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
151,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001


### GST Data

In [33]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate2 = "2013-05-01"
endDate2   = "2024-05-01"

load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

# Build URL for GST
GST_query_url= f"{base_url}{GST}?startDate2={startDate2}&endDate2={endDate2}&api_key={NASA_API_KEY}"

In [35]:
GST_query_url

'https://api.nasa.gov/DONKI/GST?startDate2=2013-05-01&endDate2=2024-05-01&api_key=c2qqOHAKmTAnSAT6wbtagaKlDS9hgW6vAJ1d2LWq'

In [37]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response=requests.get(GST_query_url)

In [38]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json=gst_response.json()
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json[:1],indent=4))

[
    {
        "gstID": "2024-09-12T09:00:00-GST-001",
        "startTime": "2024-09-12T09:00Z",
        "allKpIndex": [
            {
                "observedTime": "2024-09-12T12:00Z",
                "kpIndex": 5.67,
                "source": "NOAA"
            },
            {
                "observedTime": "2024-09-12T15:00Z",
                "kpIndex": 7.0,
                "source": "NOAA"
            },
            {
                "observedTime": "2024-09-12T18:00Z",
                "kpIndex": 6.33,
                "source": "NOAA"
            },
            {
                "observedTime": "2024-09-12T21:00Z",
                "kpIndex": 6.67,
                "source": "NOAA"
            }
        ],
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/33346/-1",
        "linkedEvents": [
            {
                "activityID": "2024-09-10T00:23:00-CME-001"
            },
            {
                "activityID": "2024-09-12T02:53:00-IPS-001"
         

In [41]:
# Convert gst_json to a Pandas DataFrame  
gst_df=pd.DataFrame(gst_json)
# Keep only the columns: activityID, startTime, linkedEvents
gst_df=gst_df[['gstID','startTime','linkedEvents']]
gst_df.head()

,gstID,startTime,linkedEvents
0,2024-09-12T09:00:00-GST-001,2024-09-12T09:00Z,[{'activityID': '2024-09-10T00:23:00-CME-001'}...
1,2024-09-13T21:00:00-GST-001,2024-09-13T21:00Z,[{'activityID': '2024-09-13T18:05:00-IPS-001'}]
2,2024-09-17T00:00:00-GST-001,2024-09-17T00:00Z,[{'activityID': '2024-09-14T15:36:00-CME-001'}...
3,2024-10-07T12:00:00-GST-001,2024-10-07T12:00Z,[{'activityID': '2024-10-03T12:48:00-CME-001'}...
4,2024-10-10T15:00:00-GST-001,2024-10-10T15:00Z,[{'activityID': '2024-10-09T02:12:00-CME-001'}...


In [43]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df.dropna(subset='linkedEvents',inplace=True)
gst_df.head()

,gstID,startTime,linkedEvents
0,2024-09-12T09:00:00-GST-001,2024-09-12T09:00Z,[{'activityID': '2024-09-10T00:23:00-CME-001'}...
1,2024-09-13T21:00:00-GST-001,2024-09-13T21:00Z,[{'activityID': '2024-09-13T18:05:00-IPS-001'}]
2,2024-09-17T00:00:00-GST-001,2024-09-17T00:00Z,[{'activityID': '2024-09-14T15:36:00-CME-001'}...
3,2024-10-07T12:00:00-GST-001,2024-10-07T12:00Z,[{'activityID': '2024-10-03T12:48:00-CME-001'}...
4,2024-10-10T15:00:00-GST-001,2024-10-10T15:00Z,[{'activityID': '2024-10-09T02:12:00-CME-001'}...


In [45]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_exploded_df=gst_df.explode('linkedEvents')
gst_exploded_df

,gstID,startTime,linkedEvents
0,2024-09-12T09:00:00-GST-001,2024-09-12T09:00Z,{'activityID': '2024-09-10T00:23:00-CME-001'}
0,2024-09-12T09:00:00-GST-001,2024-09-12T09:00Z,{'activityID': '2024-09-12T02:53:00-IPS-001'}
0,2024-09-12T09:00:00-GST-001,2024-09-12T09:00Z,{'activityID': '2024-09-12T18:40:00-MPC-001'}
1,2024-09-13T21:00:00-GST-001,2024-09-13T21:00Z,{'activityID': '2024-09-13T18:05:00-IPS-001'}
2,2024-09-17T00:00:00-GST-001,2024-09-17T00:00Z,{'activityID': '2024-09-14T15:36:00-CME-001'}
2,2024-09-17T00:00:00-GST-001,2024-09-17T00:00Z,{'activityID': '2024-09-16T22:49:00-IPS-001'}
3,2024-10-07T12:00:00-GST-001,2024-10-07T12:00Z,{'activityID': '2024-10-03T12:48:00-CME-001'}
3,2024-10-07T12:00:00-GST-001,2024-10-07T12:00Z,{'activityID': '2024-10-06T06:55:00-IPS-001'}
3,2024-10-07T12:00:00-GST-001,2024-10-07T12:00Z,{'activityID': '2024-10-07T22:12:00-IPS-001'}
4,2024-10-10T15:00:00-GST-001,2024-10-10T15:00Z,{'activityID': '2024-10-09T02:12:00-CME-001'}


In [47]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
gst_exploded_df['CME_ActivityID']=gst_exploded_df['linkedEvents'].apply(lambda x:extract_activityID_from_dict(x))
# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_exploded_df.dropna(subset='CME_ActivityID',inplace=True)
gst_exploded_df.head()

,gstID,startTime,linkedEvents,CME_ActivityID
0,2024-09-12T09:00:00-GST-001,2024-09-12T09:00Z,{'activityID': '2024-09-10T00:23:00-CME-001'},2024-09-10T00:23:00-CME-001
0,2024-09-12T09:00:00-GST-001,2024-09-12T09:00Z,{'activityID': '2024-09-12T02:53:00-IPS-001'},2024-09-12T02:53:00-IPS-001
0,2024-09-12T09:00:00-GST-001,2024-09-12T09:00Z,{'activityID': '2024-09-12T18:40:00-MPC-001'},2024-09-12T18:40:00-MPC-001
1,2024-09-13T21:00:00-GST-001,2024-09-13T21:00Z,{'activityID': '2024-09-13T18:05:00-IPS-001'},2024-09-13T18:05:00-IPS-001
2,2024-09-17T00:00:00-GST-001,2024-09-17T00:00Z,{'activityID': '2024-09-14T15:36:00-CME-001'},2024-09-14T15:36:00-CME-001


In [51]:
# Convert the 'CME_ActivityID' column to string format 
gst_exploded_df['CME_ActivityID']=gst_exploded_df['CME_ActivityID'].astype(str)
# Convert the 'gstID' column to string format 
gst_exploded_df['gstID']=gst_exploded_df['gstID'].astype(str)
# Convert startTime to datetime format  
gst_exploded_df['startTime']=pd.to_datetime(gst_exploded_df['startTime'])
# Rename startTime to startTime_GST 
gst_exploded_df.rename(columns={'startTime':'startTime_GST','activityID':'cmeID'},inplace=True)
# Drop linkedEvents
gst_exploded_df.drop(columns='linkedEvents',inplace=True)
# Verify that all steps were executed correctly
gst_exploded_df.head()

,gstID,startTime_GST,CME_ActivityID
0,2024-09-12T09:00:00-GST-001,2024-09-12 09:00:00+00:00,2024-09-10T00:23:00-CME-001
0,2024-09-12T09:00:00-GST-001,2024-09-12 09:00:00+00:00,2024-09-12T02:53:00-IPS-001
0,2024-09-12T09:00:00-GST-001,2024-09-12 09:00:00+00:00,2024-09-12T18:40:00-MPC-001
1,2024-09-13T21:00:00-GST-001,2024-09-13 21:00:00+00:00,2024-09-13T18:05:00-IPS-001
2,2024-09-17T00:00:00-GST-001,2024-09-17 00:00:00+00:00,2024-09-14T15:36:00-CME-001


In [55]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
gst_exploded_df=gst_exploded_df[gst_exploded_df['CME_ActivityID'].str.contains('CME')]
gst_exploded_df.head()

,gstID,startTime_GST,CME_ActivityID
0,2024-09-12T09:00:00-GST-001,2024-09-12 09:00:00+00:00,2024-09-10T00:23:00-CME-001
2,2024-09-17T00:00:00-GST-001,2024-09-17 00:00:00+00:00,2024-09-14T15:36:00-CME-001
3,2024-10-07T12:00:00-GST-001,2024-10-07 12:00:00+00:00,2024-10-03T12:48:00-CME-001
4,2024-10-10T15:00:00-GST-001,2024-10-10 15:00:00+00:00,2024-10-09T02:12:00-CME-001


### Merge both datatsets

In [67]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df=pd.merge(expanded_rows_df,gst_exploded_df,left_on=gst_exploded_df[['gstID','CME_ActivityID']],right_on=expanded_rows_df[['cmeID','GST_activityID']])
merged_df.head()

KeyError: "['GST_activityID'] not in index"

In [25]:
# Verify that the new DataFrame has the same number of rows as cme and gst


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           67 non-null     string             
 1   startTime_GST   67 non-null     datetime64[ns, UTC]
 2   CME_ActivityID  67 non-null     object             
 3   cmeID           67 non-null     object             
 4   startTime_CME   67 non-null     datetime64[ns, UTC]
 5   GST_ActivityID  67 non-null     string             
dtypes: datetime64[ns, UTC](2), object(2), string(2)
memory usage: 3.3+ KB


### Computing the time it takes for a CME to cause a GST

In [26]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.


,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID,timeDiff
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001,4 days 06:36:00
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001,2 days 04:20:00
2,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001,3 days 00:48:00
3,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001,2 days 12:45:00
4,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001,2 days 01:35:00


In [27]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 


,timeDiff
count,67
mean,2 days 21:35:13.432835820
std,1 days 00:02:46.681279427
min,1 days 05:36:00
25%,2 days 03:12:00
50%,2 days 17:48:00
75%,3 days 12:17:00
max,6 days 03:00:00


### Exporting data in csv format

In [28]:
# Export data to CSV without the index
